In [1]:
# This is Main function.
# Extracting streaming data from Twitter, pre-processing, and loading into MySQL
import credentials # Import api/access_token keys from credentials.py
import settings # Import related setting constants from settings.py 

import re
import tweepy
import mysql.connector
import pandas as pd
from textblob import TextBlob
# Streaming With Tweepy 
# http://docs.tweepy.org/en/v3.4.0/streaming_how_to.html#streaming-with-tweepy


# Override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
    '''
    Tweets are known as “status updates”. So the Status class in tweepy has properties describing the tweet.
    https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html
    '''
    
    def on_status(self, status):
        '''
        Extract info from tweets
        '''
        
        if status.retweeted:
            # Avoid retweeted info, and only original tweets will be received
            return True
        # Extract attributes from each tweet
        id_str = status.id_str
        created_at = status.created_at
        text = deEmojify(status.text)    # Pre-processing the text  
        sentiment = TextBlob(text).sentiment
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        
        user_created_at = status.user.created_at
        user_location = deEmojify(status.user.location)
        user_description = deEmojify(status.user.description)
        user_followers_count =status.user.followers_count
        longitude = None
        latitude = None
        if status.coordinates:
            longitude = status.coordinates['coordinates'][0]
            latitude = status.coordinates['coordinates'][1]
            
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        
        print(status.text)
        print("Long: {}, Lati: {}".format(longitude, latitude))
        
        # Store all data in MySQL
        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(settings.TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, \
                user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql, val)
            mydb.commit()
            mycursor.close()
    
    
    def on_error(self, status_code):
        '''
        Since Twitter API has rate limits, stop srcraping data as it exceed to the thresold.
        '''
        if status_code == 420:
            # return False to disconnect the stream
            return False


In [2]:
def clean_tweet(self, tweet): 
    ''' 
    Use sumple regex statemnents to clean tweet text by removing links and special characters
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 
def deEmojify(text):
    '''
    Strip all non-ASCII characters to remove emoji characters
    '''
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [3]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="mysqlroot",
    database="TwitterDB",
    charset = 'utf8'
)
if mydb.is_connected():
    '''
    Check if this table exits. If not, then create a new one.
    '''
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(settings.TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(settings.TABLE_NAME, settings.TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()
    



In [4]:
auth  = tweepy.OAuthHandler(credentials.API_KEY, credentials.API_SECRET_KEY)
auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [5]:
print(credentials.API_KEY)

CNFjjRaG9SKaEffnnsEleInVw


In [6]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
myStream.filter(languages=["en"], track = settings.TRACK_WORDS)
# Close the MySQL connection as it finished
# However, this won't be reached as the stream listener won't stop automatically
# Press STOP button to finish the process.
mydb.close()

@RepKinzinger If it’s a fabrication then go ask Facebook and Twitter why they are banning Qanon accounts by the hun… https://t.co/LBTWEmwPBv
Long: None, Lati: None
RT @forensictoxguy: I just got kicked out of a flat earth Facebook group because I asked if the 6 foot social distancing guideline had push…
Long: None, Lati: None
RT @myboxerluvmac1: Boutiful Burt was adopted yesterday !! 🎉❤🎉❤ https://t.co/f31gpo6cQp
Long: None, Lati: None
RT @Lindaakoto6: I'm watching Erica on
showmax, dstv, Netflix,
Instagram, Twitter, Facebook n snapchat.
I pray u carry 🏋️‍♂️ #BBNaija till…
Long: None, Lati: None
RT @MarkDice: Facebook gave me a "false information" strike because I posted this picture of John Lewis's funeral and noted many attendees…
Long: None, Lati: None
Thunderstorm in Cardiff
@Ruth_ITV https://t.co/qgMiHJBlOz
Long: None, Lati: None
RT @crestsac: Pretty incredible the outpouring of support by everyone. We are at almost 900 donors. Retweet and help us get there. https://…
Long: None, L

RT @adamhousley: My Seattle friends are posting some crazy stuff on facebook. Things they are seeing and hearing and they aren’t happy abou…
Long: None, Lati: None
RT @MarkDice: Facebook gave me a "false information" strike because I posted this picture of John Lewis's funeral and noted many attendees…
Long: None, Lati: None
RT @350MassAct: Tune in to see 350 Mass Action endorsed candidate @JordanMeehan at the Seventeenth Suffolk District Candidate Forum tonight…
Long: None, Lati: None
My brother posted one of those Facebook threads like “post something silver in your camera roll” but the only silve… https://t.co/kxZB7E8D37
Long: None, Lati: None
@judybattista SWEDEN had it right. No lockdowns, no masks and now they have 100% herd immunity. Our medical leaders… https://t.co/sZG6hKo3e2
Long: None, Lati: None
Women of Worth ...

Join us Thursday for a focused time of prayer and fasting. 

God is moving on our behalf! Mirac… https://t.co/SuBgUSlHAf
Long: None, Lati: None
RT @robbystarbuck

Thanks Deathsantis @GovRonDeSantis
Long: None, Lati: None
RT @desimojito: Naseeruddin Shah, Aamir Khan, Javed Akhtar and Javed Jaffrey must be feeling safe now and believe that India is tolerant af…
Long: None, Lati: None
So it’s been a while but someone on Facebook today reopened the old “you look like Shia” file

I dunno, I guess may… https://t.co/KmEFLGfrGz
Long: None, Lati: None
@Juks_Afrika thank you for following back! Please also like our FB page: https://t.co/O8usK9ImdF
Long: None, Lati: None
Kamala Harris Doesn’t Think You Have The Right To Own A Gun
https://t.co/Mdrf27NKPn
Long: None, Lati: None
Can i get some RTs and signal boosting?
Long: None, Lati: None
RT @robbystarbuck: Heads of comms at Twitter, Facebook, Instagram and Snapchat all worked for elected Democrats previously and in some case…
Long: None, Lati: None
This by @barrymward is a terrific example of how to dismantle Sinn Fein rhetoric.
Long: None, Lati: None
RT @DjWatkinss: This is panic. Shooting at a helicopter

Not me getting an update from the Facebook group me and my ex used to share lmfao what
Long: None, Lati: None
"Google, Facebook and Others Form Tech Coalition to Secure U.S. Election" by BY MIKE ISAAC AND KATE CONGER via NYT… https://t.co/VRfcWTNo6r
Long: None, Lati: None
Continued on our Facebook page at Saints Community Church in -Chris Onayinka Ministries https://t.co/2skBDFgc7S
Long: None, Lati: None
After a Year of Investigation, the Border Patrol Has Little to Say About Agents’ Misogynistic and Racist Facebook G… https://t.co/WixAvA8fT6
Long: None, Lati: None
@brianstelter And Facebook went dead at the mention of Charoltesville.

Can't wait hear Zuckerbergs excuse.
Long: None, Lati: None
RT @mvmeet: When Hindus got angry over jokes on Hindu deities,

Few Bollywood celebs said "Ashamed to be a Hindu"!

Now that entire Bangalo…
Long: None, Lati: None
RT @Tennessean: "Nashville is a special place – and it is a community we want to be part of," writes Facebook executive Sheryl Sandbe

RT @UnknownBreed__: Ppl rlly be on Facebook justifying smoking WEED while pregnant.... like huh???? 😭
Long: None, Lati: None
RT @uttaran: @khanumarfa Indian secularism is defined by potential muslim terrorists in every lane:
https://t.co/zdXnLiMapZ
Long: None, Lati: None
RT @VDAREJamesK: This inspiring and brave Woman of Color has given us the means of a compromise. Woke Capital can pay reparations. After al…
Long: None, Lati: None
RT @BambamClothes: RESPECT FOR HUMAN RIGHTS.
#FightwithPanusaya #SavePanusaya 
#หยุดคุกคามประชาชน https://t.co/2u0EeoyTwg https://t.co/JLgE…
Long: None, Lati: None
RT @SantaSurfing17: President Trump's ad that got banned....
Thank you  @PatriotRodSteel 
Thank you Mr. President! https://t.co/VTjEfXvQWH
Long: None, Lati: None
RT @prachatai_en: Thammasat University Student Union posted on their Facebook page that student security officers have gone to the student…
Long: None, Lati: None
RT @jsstansel: Hashtags on Facebook:

Completely useless 
or 
Incredibly po

...

Facebook Removes Over 1,000 Ad Targeting Options Due to Low Usage https://t.co/weN0iEOWbe https://t.co/4ggt6BcNQz
Long: None, Lati: None
Facebook Removes Over 1,000 Ad Targeting Options Due to Low Usage https://t.co/2Y4nUwcwqm https://t.co/3DjtNTpHKJ
Long: None, Lati: None
@RawBeautyKristi I can honestly say that Facebook is the only social media platform that makes me feel stressed and… https://t.co/jV5EOTK47e
Long: None, Lati: None
Facebook will show pop-ups before you share articles about COVID-19 https://t.co/mADRCBc7pf #TechNews https://t.co/fnm5nBBN3d
Long: None, Lati: None
RT @Wat_the_deuce: Why are journalists not asking questions? Or helping this mother? @LeonaONeill1 @KevinPMeagher @BJPMarshall 

https://t.…
Long: None, Lati: None
Facebook Adds New Informational Prompt When Users Share COVID-19 Related Content https://t.co/ffnmpftUsV https://t.co/OhQtWXQDoh
Long: None, Lati: None
Facebook Adds New Informational Prompt When Users Share COVID-19 Related Content https://t.co/9qV

RT @mgbelka: Meanwhile, in Bend, people are blockading two ICE prisoner transport vehicles in the parking lot of a hotel. https://t.co/tp7n…
Long: None, Lati: None
@SallyAlbright @TravisBShaw17 @TexileOrozco65 @adamhousley I won't mark them up extra it's not the right thing. I d… https://t.co/OBJes6sVxh
Long: None, Lati: None
I typically don’t respond to trolls on Facebook but I’m bored and it’s so easy to get them angry by pointing out ho… https://t.co/8zWyHNWh2d
Long: None, Lati: None
And this is his A-game

https://t.co/U6wqGE88o5
Long: None, Lati: None
@girl_kaybee follow back &amp; pease also like our FB page: https://t.co/WGQAdZLZKr
Long: None, Lati: None
RT @MarkDice: Facebook gave me a "false information" strike because I posted this picture of John Lewis's funeral and noted many attendees…
Long: None, Lati: None
RT @AngrierWHStaff: I missed this one the other day: 

Facebook removes hundreds of foreign troll farm members who were  posing as African…
Long: None, Lati: None
We’v

RT @Baby95Sage: It’s trending on Facebook, now to take the hashtag to raise awareness to all the elites on Twitter that we know what’s goin…
Long: None, Lati: None
The Donald Trump base has resurfaced.  https://t.co/u59evOCzo7
Long: None, Lati: None
RT @mgbelka: Meanwhile, in Bend, people are blockading two ICE prisoner transport vehicles in the parking lot of a hotel.
Long: None, Lati: None
RT @bokafecs: Here's a link to donate to!
➡ https://t.co/nBxjDXqGbq https://t.co/W3MKlrrixs
Long: None, Lati: None
#NowPlaying The Dillinger Escape Plan - Milk Lizard #listen https://t.co/Ngu5vVpo0O and https://t.co/yVYWHGGoJy… https://t.co/3EmdfYoClf
Long: None, Lati: None
RT @FCIACB: This article  is written on the mastermind of Mafia clan @khatir_ahmadzai who is one of the candidates for CEO position.
@ICC
@…
Long: None, Lati: None
RT @fatimaderay: I N F I N I T E 

Grab yours now visit https://t.co/awB5LOiHdf https://t.co/bxlrQjF3uj
Long: None, Lati: None
RT @LeoonD: This is great- @mrjamesob  

RT @MissingPetsGB: 🐑BORIS HAS BEEN STOLEN

HE IS JUST A BABY 1YR OLD-HOW COULD YOU?

£500 REWARD FOR HIS SAFE RETURN!

White Male Boer, ear…
Long: None, Lati: None
RT @JulieReichwein1: Laura Loomer exposes CAIR(Hamas) &amp; Kama1a Harr1s who charged Mark Feigin, FALSELY, 4 hate charges because he posted a…
Long: None, Lati: None
RT @MarkDice: Facebook gave me a "false information" strike because I posted this picture of John Lewis's funeral and noted many attendees…
Long: None, Lati: None
@Igaytrips Thanks for the follow. Like us on Facebook https://t.co/7LYFx9wTrk Visit our site https://t.co/W8wkC7rDqo
Long: None, Lati: None
RT @RepAdams: Please join us at 6:00 PM tonight for an important conversation on economic mobility! We'll also be live on Facebook.
 
Regis…
Long: None, Lati: None
@nburdy For reference: 

https://t.co/WAjdF76iR3
Long: None, Lati: None
RT @ThePerfumeShop: Enter to be in with a chance to WIN a Celeb Fragrance! ✨
1. Like &amp; Retweet
2. Comment #WIN 

Ends 23:59pm,

KeyboardInterrupt: 